# Case I/O

AMS supprots multiple case formats.

Still, first import the `ams` library and configure the logger level.

In [1]:
import os

import ams

In [2]:
ams.config_logger(stream_level=20)

## Input

### AMS Execel

In [3]:
sp_xlsx = ams.load(ams.get_case('ieee14/ieee14_uced.xlsx'),
                   setup=True,
                   no_output=True,)

sp_xlsx.summary()

Parsing input file "/Users/jinningwang/work/ams/ams/cases/ieee14/ieee14_uced.xlsx"...
Input file parsed in 0.0644 seconds.
All bus type are PQ, adjusted given load and generator connection status.
System set up in 0.0023 seconds.
-> Systen size:
Base: 100 MVA; Frequency: 60 Hz
14 Buses; 20 Lines; 5 Static Generators
Active load: 2.24 p.u.; Reactive load: 0.95 p.u.
-> Data check results:
DCED: DCOPF, DCOPF2, ED, RTED
DCUC: UC
DED: DOPF
PF: DCPF, PFlow


### AMS JSON

In [4]:
sp_json = ams.load(ams.get_case('ieee14/ieee14.json'),
                   setup=True,
                   no_output=True,)

sp_json.summary()

Parsing input file "/Users/jinningwang/work/ams/ams/cases/ieee14/ieee14.json"...
Input file parsed in 0.0013 seconds.
Zero Line parameters detected, adjusted to default values: rate_c.
All bus type are PQ, adjusted given load and generator connection status.
System set up in 0.0024 seconds.
-> Systen size:
Base: 100 MVA; Frequency: 60 Hz
14 Buses; 20 Lines; 5 Static Generators
Active load: 2.24 p.u.; Reactive load: 0.95 p.u.
-> Data check results:
PF: DCPF, PFlow


### MATPOWER

In [5]:
help(ams.io.matpower.mpc2system)

Help on function mpc2system in module ams.io.matpower:

mpc2system(mpc: dict, system) -> bool
    Load an mpc dict into an empty AMS system.

    Revised from ``andes.io.matpower.mpc2system``.

    Note that `mbase` in mpc is converted to `Sn`, but it is not actually used in
    MATPOWER nor AMS.

    In converted AMS system, StaticGen idxes are 1-based, while the sequence follow
    the order of the original MATPOWER data.

    Parameters
    ----------
    system : ams.system.System
        Empty system to load the data into.
    mpc : dict
        mpc struct names : numpy arrays

    Returns
    -------
    bool
        True if successful, False otherwise.



In [6]:
sp_mp = ams.load(ams.get_case('matpower/case14.m'),
                 setup=True,
                 no_output=True,)

sp_mp.summary()

Parsing input file "/Users/jinningwang/work/ams/ams/cases/matpower/case14.m"...
CASE14    Power flow data for IEEE 14 bus test case.
Input file parsed in 0.0032 seconds.
Zero Line parameters detected, adjusted to default values: rate_a, rate_b, rate_c.
System set up in 0.0017 seconds.
-> Systen size:
Base: 100.0 MVA; Frequency: 60 Hz
14 Buses; 20 Lines; 5 Static Generators
Active load: 2.59 p.u.; Reactive load: 0.74 p.u.
-> Data check results:
DCED: DCOPF, DCOPF2
DED: DOPF
PF: DCPF, PFlow


Note that AMS also supports PYPOWER format py-file.

### PSS/E RAW

AMS also supports PSS/E v32/v33 RAW format for power flow analysis.

In [7]:
sp_raw = ams.load(ams.get_case('ieee14/ieee14.raw'),
                  setup=True,
                  no_output=True,)

sp_raw.summary()

Parsing input file "/Users/jinningwang/work/ams/ams/cases/ieee14/ieee14.raw"...
Input file parsed in 0.0043 seconds.
Zero Line parameters detected, adjusted to default values: rate_c.
All bus type are PQ, adjusted given load and generator connection status.
System set up in 0.0022 seconds.
-> Systen size:
Base: 100.0 MVA; Frequency: 60.0 Hz
14 Buses; 20 Lines; 5 Static Generators
Active load: 2.24 p.u.; Reactive load: 0.95 p.u.
-> Data check results:
PF: DCPF, PFlow


## Output

At the meanwhile, AMS supports multiple output formats.

In [8]:
ams.io.xlsx.write(system=sp_xlsx,
                  outfile='out.xlsx',)

xlsx file written to "out.xlsx"


True

In [9]:
os.remove('out.xlsx')

Similarly, JSON output formats can be achieved by using `ams.io.json.write`.

As of v1.0.10, follo wrapper methods are provided for easy converting the case to different formats:

### MATPOWER MPC

In [10]:
help(ams.system.System.to_mpc)

Help on function to_mpc in module ams.system:

to_mpc(self)
    Export an AMS system to a MATPOWER dict.
    Wrapper method for `ams.io.matpower.system2mpc`.

    Returns
    -------
    dict
        A dictionary representing the MATPOWER case.

    Notes
    -----
    - In the `gen` section, slack generators are listed before PV generators.
    - For uncontrolled generators (`ctrl.v == 0`), their max and min power
      limits are set to their initial power (`p0.v`) in the converted MPC.
    - In the converted MPC, the indices of area (`bus[:, 6]`) and zone (`bus[:, 10]`)
      may differ from the original MPC. However, the mapping relationship is preserved.
      For example, if the original MPC numbers areas starting from 1, the converted
      MPC may number them starting from 0.
    - The coefficients `c2` and `c1` in the generator cost data are scaled by
      `baseMVA`.
    - Unlike the XLSX and JSON converters, this implementation uses value providers
      (`v`) instead of vin

### MATPOWER M-file

In [11]:
help(ams.system.System.to_m)

Help on function to_m in module ams.system:

to_m(self, outfile: str, overwrite: bool = None)
    Export an AMS system to a MATPOWER M-file.
    Wrapper method for `ams.io.matpower.write`.

    Parameters
    ----------
    outfile : str
        The output file name.
    overwrite : bool, optional
        If True, overwrite the existing file. Default is None.

    Notes
    -----
    - In the `gen` section, slack generators are listed before PV generators.
    - For uncontrolled generators (`ctrl.v == 0`), their max and min power
      limits are set to their initial power (`p0.v`) in the converted MPC.
    - In the converted MPC, the indices of area (`bus[:, 6]`) and zone (`bus[:, 10]`)
      may differ from the original MPC. However, the mapping relationship is preserved.
      For example, if the original MPC numbers areas starting from 1, the converted
      MPC may number them starting from 0.
    - The coefficients `c2` and `c1` in the generator cost data are scaled by
      `bas

### AMS XLSX

In [12]:
help(ams.system.System.to_xlsx)

Help on function to_xlsx in module ams.system:

to_xlsx(self, outfile: str, overwrite: bool = None)
    Export an AMS system to an Excel file.
    Wrapper method for `ams.io.xlsx.write`.

    Parameters
    ----------
    outfile : str
        The output file name.
    overwrite : bool, optional
        If True, overwrite the existing file. Default is None.



### AMS JSON

In [13]:
help(ams.system.System.to_json)

Help on function to_json in module ams.system:

to_json(self, outfile: str, overwrite: bool = None)
    Export an AMS system to a JSON file.
    Wrapper method for `ams.io.json.write`.

    Parameters
    ----------
    outfile : str
        The output file name.
    overwrite : bool, optional
        If True, overwrite the existing file. Default is None.



### PSS/E v33 RAW

This method has not been fully benchmarked yet!

In [14]:
help(ams.system.System.to_raw)

Help on function to_raw in module ams.system:

to_raw(self, outfile: str, overwrite: bool = None)
    Export an AMS system to a v33 PSS/E RAW file.
    Wrapper method for `ams.io.psse.write_raw`.

    This method has not been fully benchmarked yet!

    Parameters
    ----------
    outfile : str
        The output file name.
    overwrite : bool, optional
        If True, overwrite the existing file. Default is None.

